# Global Daily Event Analysis: Marine Heatwave ID & Tracking using `MarEx`

### `MarEx` Processing Pipeline for Gridded Datasets:

1. **Morphological Pre-Processing**
    - Performs binary morphological closing using `dask_image.ndmorph` to fill small spatial holes up to `R_fill` cells in radius 
    - Executes binary opening to remove isolated small features of order `R_fill`
    - Fills gaps in time to maintain event continuity for interruptions up to `T_fill` time steps
    - Filters out smallest objects below the `area_filter_quartile` percentile threshold

2. **Blob Identification**
    - Labels spatially connected components using efficient connected-component algorithm in `dask_image.ndmeasure`
    - Computes blob properties (area, centroid, boundaries)

3. **Temporal Tracking**
    - Identifies blob overlaps between consecutive time frames
    - Connects objects across time, applying the following criteria for splitting, merging, & persistence:
        - Connected objects must overlap by at least fraction `overlap_threshold` of the smaller area
        - Merged objects retain their original ID, but partition the child area based on the parent of the _nearest-neighbour_ cell (or centroid distance)

4. **Graph Reduction & Finalisation**
    - Constructs the complete temporal graph of object evolution through time
    - Resolves object connectivity graph using `scipy.sparse.csgraph.connected_components`
    - Creates globally unique IDs for each tracked extreme event
    - Maps objects into efficient ID-time space for convenient analysis
    - Computes comprehensive statistics about the lifecycle of each event

The pipeline leverages **dask** for distributed parallel computation, enabling efficient processing of large datasets. \
A 40-year global daily (OSTIA) analysis at 0.25° resolution on 32 cores takes
- Basic (i.e. Scannell et al., which involves no merge/split criteria or tracking):  ~5 minutes
- Full Split/Merge Thresholding & Merge Tracking:  ~40 minutes

In [1]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [2]:
# Lustre Scratch Directory
scratch_dir = Path('/scratch') / getuser()[0] / getuser()

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=32, threads_per_worker=1,
                                 scratch_dir = scratch_dir / 'clients')  # Specify temporary scratch directory for dask to use

Dask Scratch: '/scratch/b/b382615/clients/tmp078q_84w'
Memory per Worker: 15.74 GB
Hostname: l40201
Forward Port: l40201:8787
Dashboard Link: localhost:8787/status


In [4]:
# Choose optimal chunk size & load data
#   N.B.: This is crucial for dask (not only for performance, but also to make the problem tractable)
#         The operations are eventually global-in-space, and so requires the spatial dimension to be contiguous/unchunked
#         We can adjust the chunk size in time depending on available system memory.

chunk_size = {'time': 25, 'lat': -1, 'lon': -1}

In [5]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = scratch_dir / 'mhws' / 'extremes_binary_gridded_shifting_hobday.zarr'
ds = xr.open_zarr(str(file_name), chunks=chunk_size)

In [6]:
# Run ID, Tracking, & Merging

tracker = marEx.tracker(ds.extreme_events, 
                       ds.mask.where((ds.lat < 85) &
                                     (ds.lat > -90), other=False),  # Modify Mask: Anisotropy of the lat/lon grid near the poles biases the ID & Tracking
                       area_filter_quartile = 0.5,                  # Remove the smallest 50% of the identified coherent extreme areas
                       R_fill = 8,                                  # Fill small holes with radius < 8 _cells_
                       T_fill = 2,                                  # Allow gaps of 2 days and still continue the event tracking with the same ID
                       allow_merging = True,                        # Allow extreme events to split/merge. Keeps track of merge events & unique IDs.
                       overlap_threshold = 0.5,                     # Overlap threshold for merging events. If overlap < threshold, events keep independent IDs.
                       nn_partitioning = True,                      # Use new NN method to partition merged children areas. If False, reverts to old method of Di Sun et al. 2023.
                       verbosity=1)                                 # Choose Verbosity Level (0=None, 1=Basic, 2=Advanced/Timing)

extreme_events_ds, merges_ds = tracker.run(return_merges=True)
extreme_events_ds

Finished filling spatial holes
Finished filling spatio-temporal holes
Finished filtering small objects
Finished object identification
Finished calculating object properties
Finished finding overlapping objects


/home/b/b382615/opt/anaconda3/envs/py310/lib/python3.10/site-packages/dask/array/core.py:5097: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(


Processing splitting and merging in chunk 0 of 591
Processing splitting and merging in chunk 10 of 591
Processing splitting and merging in chunk 20 of 591
Processing splitting and merging in chunk 30 of 591
Processing splitting and merging in chunk 40 of 591
Processing splitting and merging in chunk 50 of 591
Processing splitting and merging in chunk 60 of 591
Processing splitting and merging in chunk 70 of 591
Processing splitting and merging in chunk 80 of 591
Processing splitting and merging in chunk 90 of 591
Processing splitting and merging in chunk 100 of 591
Processing splitting and merging in chunk 110 of 591
Processing splitting and merging in chunk 120 of 591
Processing splitting and merging in chunk 130 of 591
Processing splitting and merging in chunk 140 of 591
Processing splitting and merging in chunk 150 of 591
Processing splitting and merging in chunk 160 of 591
Processing splitting and merging in chunk 170 of 591
Processing splitting and merging in chunk 180 of 591
Proc

<xarray.Dataset> Size: 74GB
Dimensions:       (lat: 720, lon: 1440, time: 14761, ID: 13612, component: 2,
                   sibling_ID: 10)
Coordinates:
  * lat           (lat) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) datetime64[ns] 118kB 1982-01-01T12:00:00 ... 2022-05...
  * ID            (ID) int32 54kB 1 2 3 4 5 6 ... 13608 13609 13610 13611 13612
Dimensions without coordinates: component, sibling_ID
Data variables:
    ID_field      (time, lat, lon) int32 61GB dask.array<chunksize=(25, 720, 1440), meta=np.ndarray>
    global_ID     (time, ID) int32 804MB dask.array<chunksize=(25, 13612), meta=np.ndarray>
    area          (time, ID) float32 804MB dask.array<chunksize=(25, 13612), meta=np.ndarray>
    centroid      (component, time, ID) float64 3GB dask.array<chunksize=(2, 25, 13612), meta=np.ndarray>
    presence      (time, ID) bool 201MB dask.array<chunksize=(25, 13612), meta=np.ndarray>
    time_start    (ID) datetime64[ns] 109kB dask.array<chunksize=(13612,), meta=np.ndarray>
    time_end      (ID) datetime64[ns] 109kB dask.array<chunksize=(13612,), meta=np.ndarray>
    merge_ledger  (time, ID, sibling_ID) int32 8GB dask.array<chunksize=(25, 13612, 10), meta=np.ndarray>
Attributes: (12/14)
    allow_merging:               1
    N_objects_prefiltered:       250096
    N_objects_filtered:          125109
    N_events_final:              13612
    R_fill:                      8
    T_fill:                      2
    ...                          ...
    accepted_area_fraction:      0.9250359966957292
    preprocessed_area_fraction:  0.603239018193744
    overlap_threshold:           0.5
    nn_partitioning:             1
    total_merges:                26517
    multi_parent_merges:         6680

In [7]:
merges_ds

<xarray.Dataset> Size: 3MB
Dimensions:        (merge_ID: 26517, parent_idx: 10, child_idx: 10)
Dimensions without coordinates: merge_ID, parent_idx, child_idx
Data variables:
    parent_IDs     (merge_ID, parent_idx) int32 1MB 2 5 -1 -1 -1 ... -1 -1 -1 -1
    child_IDs      (merge_ID, child_idx) int32 1MB 15 125110 -1 -1 ... -1 -1 -1
    overlap_areas  (merge_ID, parent_idx) int32 1MB 4790 660 -1 -1 ... -1 -1 -1
    merge_time     (merge_ID) datetime64[ns] 212kB 1997-01-02T12:00:00 ... 20...
    n_parents      (merge_ID) int8 27kB 2 2 2 3 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2
    n_children     (merge_ID) int8 27kB 2 2 2 3 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2
Attributes:
    fill_value:  -1

In [8]:
# Save IDed/Tracked/Merged Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / 'mhws' / 'extreme_events_merged_gridded_shifting.zarr'
extreme_events_ds.to_zarr(file_name, mode='w')

In [9]:
# Save Merges Dataset to netcdf
file_name = scratch_dir / 'mhws' / 'extreme_events_merged_gridded_shifting_merges.nc'
merges_ds.to_netcdf(file_name, mode='w')

### Run Basic Tracking for Comparison
N.B.: This is the current standard method used in the literature, which involves _No_ temporal gap filling, _No_ merging/splitting and _No_ independent event tracking.

In [ ]:
# Run Basic Tracking

tracker = marEx.tracker(ds.extreme_events, 
                       ds.mask.where((ds.lat < 85) &
                                     (ds.lat > -90), other=False),  # Modify Mask: Anisotropy of the lat/lon grid near the poles biases the ID & Tracking
                       area_filter_quartile = 0.5,                  # Remove the smallest 50% of the identified coherent extreme areas
                       R_fill = 8,                                  # Fill small holes with radius < 8 _cells_
                       T_fill = 0,                                  # No temporal hole filling
                       allow_merging = False,                       # Do not allow extreme events to split/merge. All touching events adopt the same ID forever (after _and_ before (!)).
                       verbosity=1)

extreme_events_basic_ds = tracker.run()
extreme_events_basic_ds

Finished filling spatial holes
Finished filling spatio-temporal holes


: 

In [ ]:
# Save IDed Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / 'mhws' / 'extreme_events_basic_gridded_shifting.zarr'
extreme_events_basic_ds.to_zarr(file_name, mode='w')